# Regularization with Ridge and Lasso

This notebook visualizes the effect of ridge and lasso on the wine dataset.

---

### 1. Load standard packages

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

---

### 2. Import data

Load in the wine dataset with pandas. This version has already had the red and white wines concatenated together and tagged with a binary 1,0 indicator (1 is red wine).

In [ ]:
wine = pd.read_csv('../assets/datasets/winequality_merged.csv')

# replace spaces in column names and convert all columns to lowercase:
wine.columns = [x.lower().replace(' ','_') for x in wine.columns]

In [ ]:
wine.head()

---

### 3. Normalize the predictor columns

With the Lasso and Ridge it is very neccessary to normalize the predictor columns before constructing the models, even the dummy coded categorical variables. Below we define our target variable and then normalize the columns that are not the target.

**Q. Can you explain why normalization is so important in the context of regularization?**

In [ ]:
# Choose a target/dependent variable that we will predict
target = 'quality'

In [ ]:
# select all the columns that are not the target
nc = [x for x in wine.columns if x != target]

# Using the .ix indexing syntax, subtract mean and divide by standard deviation for all predictor columns.
#
# Remember: .ix indexing notation works like: data.ix[row_indices, column_indices]
# .ix is able to take a mix of boolean, number, or string specifications, which is useful.
#
# By subtracting the mean and dividing by the standard devation, the normalization procedure is putting 
# all of the predictor variables on the same scale (distributions with mean == 0 and standard deviation == 1)
wine.ix[:, nc] = (wine.ix[:, nc] - wine.ix[:, nc].mean()) / wine.ix[:, nc].std()

In [ ]:
wine.head()

---

### 4. Check out the correlation matrix

Correlation matrix is useful to get some intuition right away for the relationships (or lack thereof) between all of our variables.

Print out the numeric correlation. Also make a heatmap with seaborn.

In [ ]:
wine_corr = wine.corr()
wine_corr

In [ ]:
# Set the default matplotlib figure size to 7x7:
plt.rcParams['figure.figsize']=(9,7)

# Generate a mask for the upper triangle (taken from seaborn example gallery)
mask = np.zeros_like(wine_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Plot the heatmap with seaborn.
# Assign the matplotlib axis the function returns. This will let us resize the labels.
ax = sns.heatmap(wine_corr, mask=mask)

# Resize the labels.
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=14)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=14)

# If you put plt.show() at the bottom, it prevents those useless printouts from matplotlib.
plt.show()

---

### 5. Make X and Y data

Import patsy so we can use the formula syntax.

We will be making two X, Y datasets. 

1. The first is going to just be all the variables added together.
2. The second will 'multiply' the variables together. This means that every possible interaction between variables will be modeled, which should overfit the target quite a bit!


In [ ]:
import patsy

# Get the non-target cols with a simple list comprehension
non_target_cols = [c for c in wine.columns if c != target]

# Use some string adding and joining to make the simple model formula:
formula_simple = target + ' ~ ' + ' + '.join(non_target_cols) + ' -1'
print formula_simple

# Make the complex formula:
formula_complex = target + ' ~ (' + ' + '.join(non_target_cols) + ')**2 -1'
print formula_complex

# Create the X and Y pairs for both!
Y, X = patsy.dmatrices(formula_simple, data=wine)
Yoverfit, Xoverfit = patsy.dmatrices(formula_complex, data=wine)


Look at how the shapes of the X matrices differ. You can see taht the Xoverfit has a ton of columns due to the creation of all possible interactions.

In [ ]:
print X.shape
print Xoverfit.shape

Assign the feature names for the simple and complex models to variables. We can use the `.design_info.column_names` attribute of these patsy matrix objects.

You can print out the complex feature names if you feel like it. There are a ton.

In [ ]:
simple_feature_names = X.design_info.column_names
complex_feature_names = Xoverfit.design_info.column_names

---

### 6. Visualize the Ridge and Lasso regularization on coefficients

We will only do this for the simple data since the model that will overfit has too many coefficients to see anything properly on a graph!

#### 6.1 Visualizing the Ridge

Import the `Ridge` model class (a "blueprint" for the model) from sklearn:

In [ ]:
from sklearn.linear_model import Ridge

Here I write a function that iterates over a series of different alpha regularization parameters. The alpha is related to the lambda value that multiples the square of betas from the equation (though it may not be exactly the same numerically – regardless, increasing the alpha increases the regularization of the ridge on the coefficients).

In [ ]:
def ridge_coefs(X, Y, alphas):
    
    # set up the list to hold the different sets of coefficients:
    coefs = []
    
    # Set up a ridge regression object
    ridge_reg = Ridge()
    
    # Iterate through the alphas fed into the function:
    for a in alphas:
        
        # On each alpha reset the ridge model's alpha to the current one:
        ridge_reg.set_params(alpha=a)
        
        # fit or refit the model on the provided X, Y
        ridge_reg.fit(X, Y)
        
        # Get out the coefficient list (first element of the .coef_ attribute)
        coefs.append(ridge_reg.coef_[0])
        
    return coefs

Alpha values for the ridge are best visualized on a logarithmic "magnitude" scale. Essentially the effect does not increase linearly so much as by the increase in orders of magnitude. You will see that this is different for the lasso (and is related to the fact that for the Ridge the beta coefficients are squared, rather than just the absolute value like in the Lasso).

In [ ]:
# np.logspace gives us points between specified orders of magnitude on a logarithmic scale. It is base 10.
r_alphas = np.logspace(0, 5, 200)

# Get the coefficients for each alpha for the Ridge, using the function above
r_coefs = ridge_coefs(X, Y, r_alphas)

---

Below I've written a plotting function that will:

 - Plot the effect of changing alpha on the coefficient size on a **path** graph
 - Plot the effect of changing alpha on the coefficient size on a **bar** graph
 
Each one gives informative information. It's just two different ways of visualizing the same thing. The chart is interactive so you can play around with the values of alpha across the specified range above.

In [ ]:
# The cycler package lets us "cycle" throug colors.
# Just another thing i had to look up on stackoverflow. That's my life.
from cycler import cycler

def coef_plotter(alphas, coefs, feature_names, to_alpha, regtype='ridge'):
    
    # Get the full range of alphas before subsetting to keep the plots from 
    # resetting axes each time. (We use these values to set static axes later).
    amin = np.min(alphas)
    amax = np.max(alphas)
    
    # Subset the alphas and coefficients to just the ones below the set limit
    # from the interactive widget:
    alphas = [a for a in alphas if a <= to_alpha]
    coefs = coefs[0:len(alphas)]
    
    # Get some colors from seaborn:
    colors = sns.color_palette("husl", len(coefs[0]))
    
    # Get the figure and reset the size to be wider:
    fig = plt.figure()
    fig.set_size_inches(18,7)

    # We have two axes this time on our figure. 
    # The fig.add_subplot adds axes to our figure. The number inside stands for:
    #[figure_rows|figure_cols|position_of_current_axes]
    ax1 = fig.add_subplot(121)
    
    # Give it the color cycler:
    ax1.set_prop_cycle(cycler('color', colors))
    
    # Print a vertical line showing our current alpha threshold:
    ax1.axvline(to_alpha, lw=2, ls='dashed', c='k', alpha=0.4)
    
    # Plot the lines of the alphas on x-axis and coefficients on y-axis
    ax1.plot(alphas, coefs, lw=2)
    
    # set labels for axes:
    ax1.set_xlabel('alpha', fontsize=20)
    ax1.set_ylabel('coefficients', fontsize=20)
    
    # If this is for the ridge, set this to a log scale on the x-axis:
    if regtype == 'ridge':
        ax1.set_xscale('log')
    
    # Enforce the axis limits:
    ax1.set_xlim([amin, amax])
    
    # Put a title on the axis
    ax1.set_title(regtype+' coef paths\n', fontsize=20)
    
    # Get the ymin and ymax for this axis to enforce it to be the same on the 
    # second chart:
    ymin, ymax = ax1.get_ylim()

    # Add our second axes for the barplot in position 2:
    ax2 = fig.add_subplot(122)
    
    # Position the bars according to their index from the feature names variable:
    ax2.bar(range(1, len(feature_names)+1), coefs[-1], align='center', color=colors)
    ax2.set_xticks(range(1, len(feature_names)+1))
    
    # Reset the ticks from numbers to acutally be the names:
    ax2.set_xticklabels(feature_names, rotation=65, fontsize=12)
    
    # enforce limits and add titles, labels
    ax2.set_ylim([ymin, ymax])
    ax2.set_title(regtype+' predictor coefs\n', fontsize=20)
    ax2.set_xlabel('coefficients', fontsize=20)
    ax2.set_ylabel('alpha', fontsize=20)
    
    plt.show()


Load the ipython widgets so we can make this plotting function interactive!

In [ ]:
from ipywidgets import *
from IPython.display import display

The function and `interact` from ipywidgets lets me take some specified alphas that we have already calculated the coefficients for and plot them out.

In [ ]:
def ridge_plot_runner(log_of_alpha=0):
    coef_plotter(r_alphas, r_coefs, simple_feature_names, 10**log_of_alpha, regtype='ridge')

interact(ridge_plot_runner, log_of_alpha=(0.0,5.0,0.05))

#### 6.2 Visualizing the Lasso

Now we do the same thing as above but for the Lasso. You will be able to see how the coefficients change differently for both.

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# This is the same as the ridge coefficient by alpha calculator
def lasso_coefs(X, Y, alphas):
    coefs = []
    lasso_reg = Lasso()
    for a in alphas:
        lasso_reg.set_params(alpha=a)
        lasso_reg.fit(X, Y)
        coefs.append(lasso_reg.coef_)
        
    return coefs


Alphas for the Lasso don't have their effects by orders of magnitude. A linear series of alphas is sufficient. Don't worry about the warning for Lasso at alpha 0.

In [ ]:
l_alphas = np.arange(0, 0.2, 0.0025)
l_coefs = lasso_coefs(X, Y, l_alphas)

Run the same plotting function above, but now with the calculated coefficients by alpha for the Lasso.

In [ ]:
def lasso_plot_runner(alpha=0):
    coef_plotter(l_alphas, l_coefs, simple_feature_names, alpha, regtype='lasso')

interact(lasso_plot_runner, alpha=(0.0,0.2,0.0025))

---

### 7. Model performance of the Ridge and Lasso on the simple data

Let's check out how the penalties affect the performance of our models.

Load in the KFold crossvalidation function from sklearn. We'll make five folds.

In [ ]:
from sklearn.cross_validation import KFold
kfolds = KFold(wine.shape[0], n_folds=5)

For both the Ridge and Lasso, iterate through their alphas and using the cross-validation folds calculate the average R2 at each regularization level:

In [ ]:
def ridge_lasso_cv_rsq(X, Y, r_alphas, l_alphas, kfolds, verbose=False):
    
    # lists to track mean R2s per alpha
    ridge_rsq = []
    lasso_rsq = []

    # initialize models
    lasso = Lasso()
    ridge = Ridge()

    print 'Lasso...'
    # iterate through Lasso alphas
    for la in l_alphas:
        
        if verbose: print la
        
        # set the current alpha to the model
        lasso.set_params(alpha=la)
        
        # keep track of fold R2s
        rsqs = []
        
        # iterate through the folds. Each iteration returns the training and
        # testing indices
        for traini, testi in kfolds:
            
            # run the current model with the subset training X and Y
            lasso.fit(X[traini], Y[traini])
            
            # append the R2 on the test set to the tracker
            rsqs.append(lasso.score(X[testi], Y[testi]))
            
        # append the mean of the R2s for this alpha to the R2 by alpha list
        lasso_rsq.append(np.mean(rsqs))

    print 'Ridge...'
    # Do the same process as above for ridge...
    for ra in r_alphas:
        
        if verbose: print ra
        
        ridge.set_params(alpha=ra)
        rsqs = []
        for traini, testi in kfolds:
            ridge.fit(X[traini], Y[traini])
            rsqs.append(ridge.score(X[testi], Y[testi]))
        ridge_rsq.append(np.mean(rsqs))
        
    return ridge_rsq, lasso_rsq
    
# Get the ridge and lasso cross-validated R2s:
ridge_rsq, lasso_rsq = ridge_lasso_cv_rsq(X, Y, r_alphas, l_alphas, kfolds)


---

Below we have a new plotting function that will track the performance of the model as the alphas increase, as measured by the mean R2s across cross-validation folds. Remember that R2 is a measure of how much variance in the target/dependent variable is explained by our predictor variables.

In [ ]:
def rsq_plotter(ridge_alphas, ridge_to_alpha, ridge_rsq,
                lasso_alphas, lasso_to_alpha, lasso_rsq):
    
    
    # Find the overall minimum and maximum alpha values for
    # the Ridge and Lasso to fix the plot axes:
    ridge_amin = np.min(ridge_alphas)
    ridge_amax = np.max(ridge_alphas)
    
    lasso_amin = np.min(lasso_alphas)
    lasso_amax = np.max(lasso_alphas)
    
    # Subet the models' alphas and rsqs according to the currently set
    # alpha limits for each (passed in from the interactive sliders)
    ridge_alphas = [a for a in ridge_alphas if a <= ridge_to_alpha]
    ridge_rsq = ridge_rsq[0:len(ridge_alphas)]
    
    lasso_alphas = [a for a in lasso_alphas if a <= lasso_to_alpha]
    lasso_rsq = lasso_rsq[0:len(lasso_alphas)]
    
    # Get some unique colors out for the Ridge R2 line, Lasso R2 line,
    # and the 'max R2 achieved' line.
    colors = sns.xkcd_palette(['windows blue', 'amber', 'faded green'])

    # We will again be plotting two axes on the same figure:
    fig = plt.figure()
    fig.set_size_inches(18,7)

    # The first subplot axes is for the ridge
    ax1 = fig.add_subplot(121)
    
    # Plot a vertical line indicating the current alpha selected
    ax1.axvline(ridge_to_alpha, lw=2, ls='dashed', c='k', alpha=0.4)
    
    # Plot a horizontal line for the maximum R2 achieved so far across alphas
    # for the Ridge
    ax1.axhline(np.max(ridge_rsq), lw=2, c=colors[2], alpha=0.8)
    
    # Plot the line tracking R2s by alpha values
    ax1.plot(ridge_alphas, ridge_rsq, lw=3, c=colors[0])
    
    # Add the axis labels
    ax1.set_xlabel('ridge alpha', fontsize=20)
    ax1.set_ylabel('ridge CV R2', fontsize=20)
    
    # Set x-axis to logarithmic scale
    ax1.set_xscale('log')
    
    # Fix the axes in place
    ax1.set_xlim([ridge_amin, ridge_amax])
    ax1.set_ylim([-0.05, 1])
    
    # set the title for the axes;
    ax1.set_title('ridge cross-val performance\n', fontsize=20)
    
    
    # Now do all of this as above for the Lasso!
    ax2 = fig.add_subplot(122)
    ax2.axvline(lasso_to_alpha, lw=2, ls='dashed', c='k', alpha=0.4)
    ax2.axhline(np.max(lasso_rsq), lw=2, c=colors[2], alpha=0.8)
    
    ax2.plot(lasso_alphas, lasso_rsq, lw=3, c=colors[0])
    
    ax2.set_xlabel('lasso alpha', fontsize=20)
    ax2.set_ylabel('lasso CV R2', fontsize=20)
    ax2.set_xlim([lasso_amin, lasso_amax])
    ax2.set_ylim([-0.05, 1])
        
    ax2.set_title('lasso cross-val performance\n', fontsize=20)
    
    plt.show()

In [ ]:
# We have our new widget where you can change both the Ridge and Lasso alphas to see how they compare:

def rsq_plot_pipe(ra, la):
    rsq_plotter(r_alphas, 10**ra, ridge_rsq, l_alphas, la, lasso_rsq)
    
w = widgets.interactive(rsq_plot_pipe, 
                        ra=widgets.FloatSlider(value=0, min=0., max=5., step=0.05, description='Ridge log10(alpha):'),
                        la=widgets.FloatSlider(value=0, min=0., max=0.2, step=0.0025, description='Lasso alpha:')
)

display(w)

---

### 8. Model performance on the complex data

Let's see what happens when we do this for the complex data that is designed to overfit.

[This function takes a long time to run by comparison, as you might expect.]

In [ ]:
overfit_ridge_rsq, overfit_lasso_rsq = ridge_lasso_cv_rsq(Xoverfit, Yoverfit, 
                                                          r_alphas, l_alphas, 
                                                          kfolds, verbose=True)
